Pytorch is easier to debug, I like it.

TODO:
- [x] prioritised experience replay, need to grab loss for each sample, and sample based on loss
- [ ] check it for my data, can it overfit?, does the normalisation make sense?
- [x] better metrics
- [ ] do cnn model
- [x] read papers
- [ ] check i'm prioristising by the right things, should lead to lowest loss
- [ ] test on cartpole

Refs: 
- implementations:
    - PPO
        - **pytorch implementation https://github.com/alexis-jacq/Pytorch-DPPO/blob/master/ppo.py**
        - tensorflow implementation https://github.com/reinforceio/tensorforce/blob/master/tensorforce/models/ppo_model.py
    - Prioritised memory
    - Other
        - http://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html#training
        - https://github.com/pytorch/examples/blob/master/reinforcement_learning/reinforce.py
- papers:
    - DPPO https://arxiv.org/pdf/1707.02286.pdf
    - PPO 
        - https://arxiv.org/abs/1707.06347
        - https://blog.openai.com/openai-baselines-ppo/
    - TRPO https://arxiv.org/abs/1502.05477

In [1]:
# plotting
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# numeric
import numpy as np
from numpy import random
import pandas as pd

# utils
from tqdm import tqdm_notebook as tqdm
from collections import Counter
import tempfile
import logging
import time
import datetime
import random

from collections import OrderedDict
from IPython.display import display
from pprint import pprint

# logging
logger = log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
logging.basicConfig()
log.info('%s logger started.', __name__)

INFO:__main__:__main__ logger started.


In [2]:
import argparse
import os
import sys
import gym
from gym import wrappers
import random
import numpy as np

import torch
import torch.optim as optim
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
import os
os.sys.path.append(os.path.abspath('.'))
%reload_ext autoreload
%autoreload 2

In [4]:
class Params():
    def __init__(self):
        # env
        self.window_length = 50
        # Model
        self.batch_size = 250
        self.lr = 3e-4
        self.gamma = 0.05
        self.gae_param = 0.95
        self.clip = 0.2 # epsilon from eq 7, default 0.2
        self.ent_coeff = 0.
        self.num_epoch = 20
        self.num_steps = 2048
        self.time_horizon = 1000000
        self.max_episode_length = 10000
        self.seed = 1

params = Params()

save_path= 'outputs/agent_portfolio-ddpo/{}_weights.pickle'.format('2017-07-21')
try:
    os.makedirs(os.path.dirname(save_path))
except OSError:
    pass
save_path

'outputs/agent_portfolio-ddpo/2017-07-21_weights.pickle'

# Memory
refs
- https://github.com/openai/baselines/blob/master/baselines/deepq/replay_buffer.py
- https://github.com/jaara/AI-blog/blob/master/Seaquest-DDQN-PER.py

In [5]:
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []

    def push(self, events):
        for event in zip(*events):
            self.memory.append(event)
            if len(self.memory)>self.capacity:
                del self.memory[0]

    def clear(self):
        self.memory = []

    def sample(self, batch_size):
        samples = zip(*random.sample(self.memory, batch_size))
        return map(lambda x: torch.cat(x, 0), samples)

In [6]:
import torch.utils.data.sampler

class PrioritisedReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []

    def push(self, events):
        # event is [states, actions, returns, advantages]
        # [1x3x5x50,1x6,1x1,1x1]
        for event in zip(*events):
            self.memory.append(event)
            if len(self.memory)>self.capacity:
                del self.memory[0]

    def clear(self):
        self.memory = []

    def sample(self, batch_size, beta=0.5):
        """
        Take a weighted sample based on advantages.
        
        Half hearted implementation of algorithm 1 from https://arxiv.org/pdf/1511.05952.pdf
        
        Better one here https://github.com/openai/baselines/blob/master/baselines/deepq/replay_buffer.py#L157
        
        Parameters
        ----------
        batch_size: int
            How many transitions to sample.
        beta: float
            To what degree to use importance weights
            (0 - minimal corrections, 1 - full correction)
            
        Returns
        ----------
        states
        actions
        returns
        advantages
        
        """
        # Sample transition
        # we want to minimise loss so
        #     weights ~ -loss (bigger the weight when the loss is lower)
        # and
        #     loss ~= -batch_advantages
        # so 
        #     weights ~ batch_advantages
        ps1 = torch.FloatTensor([m[-1].squeeze().data[0] for m in self.memory])        
        ps1 -= ps1.min()
        ps1 /= ps1.sum()
        
        # batch_returns
#         ps2 = torch.FloatTensor([m[-2].squeeze().data[0] for m in self.memory])             
#         ps2 -= ps2.min()
#         ps2 /= ps2.sum()
        
        ps = ps1#+ps2 # priority        
        ps -= ps.min() - 1.0
        P = ps/ps.sum() # normalize
        
        # Compute importance-sampling weight
        P = (len(P) * P) ** (-beta)
        w = P / P.max()
        
        # to list and remove nans
        w = w.numpy()
#         w[np.isfinite(w)==False] = 0
        w = w.tolist()
        
        idxs = torch.utils.data.sampler.WeightedRandomSampler(w, params.batch_size, replacement=False)
        
        # concatenate it into batches
        samples = zip(*[self.memory[n] for n in idxs])        
        return map(lambda x: torch.cat(x, 0), samples)

In [7]:
# # Test by making sure the sample has a higher returns than the mean
# batch_states, batch_actions, batch_returns, batch_advantages = memory.sample(200)

# ps1 = torch.FloatTensor([m[-1].squeeze().data[0] for m in memory.memory])        
# print(batch_advantages.mean(),ps1.mean())
# assert batch_advantages.mean().data[0]>ps1.mean()

# # ps2 = torch.FloatTensor([m[-2].squeeze().data[0] for m in memory.memory]) 
# # assert batch_returns.mean().data[0]>ps2.mean()


# Enviroment

In [8]:
from src.environments.portfolio import PortfolioEnv, sharpe, max_drawdown

# we want to pemute the channels a little

class PermutedPortfolioEnv(PortfolioEnv):
    def reset(self, *args, **kwargs):
        return np.transpose(super().reset(*args, **kwargs),(0,1,2))
    def step(self, *args, **kwargs):
        observation, reward, done, info = super().step(*args, **kwargs)
        observation = np.transpose(observation,(2,0,1))
        return observation, reward, done, info


df_train = pd.read_hdf('./data/poloniex_30m.hf',key='train')
env = PermutedPortfolioEnv(
    df=df_train,
    steps=128, 
    scale=True, 
    augment=0.00025, # let just overfit first,
    trading_cost=0, #0.0025, # let just overfit first,
    window_length = params.window_length,   
)
env.seed(params.seed)
env.reset().shape

df_test = pd.read_hdf('./data/poloniex_30m.hf',key='test')
env_test = PermutedPortfolioEnv(
    df=df_test,
    steps=128, 
    scale=True, 
    augment=0.00025, # let just overfit first,
    trading_cost=0, #0.0025, # let just overfit first,
    window_length = params.window_length,   
)
env_test.seed(params.seed)
env_test.reset().shape

(3, 5, 50)

# Model

In [9]:

class EIIE_CNN(nn.Module):
    """
    CNN implementation of Ensemble of Identical Independent Evaluators (EIIE).
    
    https://arxiv.org/abs/1706.10059
    """
#     def __init__(self, price_his_len, nb_actions):
    def __init__(self, inputs, outputs):
        self.inputs=inputs
        self.outputs=outputs
        super(EIIE_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 2, (1, 3))
        self.conv2 = nn.Conv2d(2, 20, (1, inputs[1] - 2))
        self.conv3 = nn.Conv2d(20, 1, (1, 1))        
        self.head  = nn.Linear(outputs[0]-1,outputs[0]) # add cash bias?
        
        self.log_std = nn.Parameter(torch.zeros(num_outputs))
        
        self.v = nn.Linear(outputs[0]-1,1)

    def forward(self, inputs):
        # actor
        x = F.relu(self.conv1(inputs))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        h = x.view(x.size(0),self.outputs[0]-1) # Flatten
        
        mu = F.softmax(self.head(h)) # action
        log_std = torch.exp(self.log_std).unsqueeze(0).expand_as(mu) # exploration param?
        v = self.v(h) # critic
        
        
        # critic
        return mu, log_std, v

In [10]:
class GenericSharedModel(nn.Module):
    def __init__(self, inputs, outputs):
        super(GenericSharedModel, self).__init__()
        num_inputs = int(np.prod(env.observation_space.shape))
        num_outputs = int(np.prod(env.action_space.shape))
        
        # hidden layer sizes
        h_size_1 = 128
        h_size_2 = 128
        
        self.fc1 = nn.Linear(num_inputs, h_size_1)
        self.fc2 = nn.Linear(h_size_1, h_size_2)
        self.mu = nn.Linear(h_size_2, num_outputs)
        self.log_std = nn.Parameter(torch.zeros(num_outputs))
        self.v = nn.Linear(h_size_2,1)
        
        for name, p in self.named_parameters():
            # init parameters
            if 'bias' in name:
                p.data.fill_(0)
            '''
            if 'mu.weight' in name:
                p.data.normal_()
                p.data /= torch.sum(p.data**2,0).expand_as(p.data)'''
        
        # mode
        self.train()

    def forward(self, inputs):
        # flatten
        inputs = inputs.view((inputs.size()[0],-1))
        
        # actor
        x = F.tanh(self.fc1(inputs))
        h = F.tanh(self.fc2(x))
        
        # the action
        mu = F.softmax(self.mu(h))
        
        # exploration multiplier
        log_std = F.sigmoid(torch.exp(self.log_std).unsqueeze(0).expand_as(mu))
        
        # critic
        v = self.v(h)
        return mu, log_std, v

# Train

In [11]:
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [12]:

# class Shared_grad_buffers():
#     def __init__(self, model):
#         self.grads = {}
#         for name, p in model.named_parameters():
#             self.grads[name+'_grad'] = torch.ones(p.size()).share_memory_()

#     def add_gradient(self, model):
#         for name, p in model.named_parameters():
#             self.grads[name+'_grad'] += p.grad.data

#     def reset(self):
#         for name,grad in self.grads.items():
#             self.grads[name].fill_(0)

class Shared_obs_stats():
    """Like batchnorm for input data"""
    def __init__(self, num_inputs):
        self.n = torch.zeros(num_inputs).share_memory_()
        self.mean = torch.zeros(num_inputs).share_memory_()
        self.mean_diff = torch.zeros(num_inputs).share_memory_()
        self.var = torch.zeros(num_inputs).share_memory_()

    def observes(self, obs):
        # observation mean var updates
        x = obs.data.squeeze()
        self.n += 1.
        last_mean = self.mean.clone()
        self.mean += (x-self.mean)/self.n
        self.mean_diff += (x-last_mean)*(x-self.mean)
        self.var = torch.clamp(self.mean_diff/self.n, min=1e-2)

    def normalize(self, inputs):
        obs_mean = Variable(self.mean.unsqueeze(0).expand_as(inputs))
        obs_std = Variable(torch.sqrt(self.var).unsqueeze(0).expand_as(inputs))
        return torch.clamp((inputs-obs_mean)/obs_std, -5., 5.)

In [13]:
def normal(x, mu, sigma_sq):
    a = (-1*(x-mu).pow(2)/(2*sigma_sq)).exp()
    b = 1/(2*sigma_sq*np.pi).sqrt()
    return a*b

In [14]:

cuda = False
torch.manual_seed(params.seed)
work_dir = mkdir('exp', 'ppo')
monitor_dir = mkdir(work_dir, 'monitor')

# env = gym.make(params.env_name)
#env = wrappers.Monitor(env, monitor_dir, force=True)

num_inputs = env.observation_space.shape[0]
num_outputs = env.action_space.shape[0]


#initialize network and optimizer
Model = GenericSharedModel
Model = EIIE_CNN
model = Model(env.observation_space.shape, env.action_space.shape)
if cuda: model.cuda()

# shared_obs_stats = Shared_obs_stats(num_inputs)
optimizer = optim.Adam(model.parameters(), lr=params.lr)
model

EIIE_CNN (
  (conv1): Conv2d(3, 2, kernel_size=(1, 3), stride=(1, 1))
  (conv2): Conv2d(2, 20, kernel_size=(1, 48), stride=(1, 1))
  (conv3): Conv2d(20, 1, kernel_size=(1, 1), stride=(1, 1))
  (head): Linear (5 -> 6)
  (v): Linear (5 -> 1)
)

In [ ]:
memory = ReplayMemory(params.num_steps)
# memory = PrioritisedReplayMemory(params.num_steps)

num_inputs = int(np.prod(env.observation_space.shape))
num_outputs = int(np.prod(env.action_space.shape))

state = env.reset()
state = Variable(torch.Tensor(state).unsqueeze(0))
done = True
episode_length = 0
reports = []

while True:
#     episode_length += 1
    episode = -1
    
    # horizon loop
    for t in range(params.time_horizon):
        infos = []
        episode_length = 0
        # Sample data from the policy
        while (len(memory.memory) < params.num_steps):
            states = []
            actions = []
            rewards = []
            values = []
            returns = []
            advantages = []
            av_reward = 0
            cum_reward = 0
            cum_done = 0
            # n steps loops
            for step in range(params.num_steps):
                #                 shared_obs_stats.observes(state)
                #                 state = shared_obs_stats.normalize(state)
                states.append(state)
                
                mu, sigma_sq, v = model(state)
                eps = torch.randn(mu.size())
                action = (mu + sigma_sq.sqrt() * Variable(eps))
                env_action = action.data.squeeze().numpy()
                state, reward, done, info = env.step(env_action)
                done = (done or episode_length >= params.max_episode_length)
                
                cum_reward += reward
                reward = max(min(reward, 1), -1)
                rewards.append(reward)
                actions.append(action)
                values.append(v)
                
                if done:
                    episode += 1
                    cum_done += 1
                    av_reward += cum_reward
                    cum_reward = 0
                    episode_length = 0
                    infos.append(info)
                    state = env.reset()
                
                state = Variable(torch.Tensor(state).unsqueeze(0))
                
                if done:
                    break
            
            # one last step
            R = torch.zeros(1, 1)
            if not done:
                _, _, v = model(state)
                R = v.data
            
            # compute returns and GAE(lambda) advantages:
            values.append(Variable(R))
            R = Variable(R)
            A = Variable(torch.zeros(1, 1))
            for i in reversed(range(len(rewards))):
                td = rewards[i] + params.gamma*values[i+1].data[0,0] - values[i].data[0,0]
                A = float(td) + params.gamma * params.gae_param * A
                advantages.insert(0, A)
                R = A + values[i]
                returns.insert(0, R)
            
            # store useful info:
            memory.push([states, actions, returns, advantages])

        # perform several epochs of optimization on the sampled data
        model_old = Model(env.observation_space.shape,
                             env.action_space.shape)
        model_old.load_state_dict(model.state_dict())
        av_loss = 0
        for k in range(params.num_epoch):
            # cf https://github.com/openai/baselines/blob/master/baselines/pposgd/pposgd_simple.py
            batch_states, batch_actions, batch_returns, batch_advantages = memory.sample(
                params.batch_size)
            
            # old probas
            mu_old, sigma_sq_old, v_pred_old = model_old(batch_states.detach())
            probs_old = normal(batch_actions, mu_old, sigma_sq_old)
            
            # new probas
            mu, sigma_sq, v_pred = model(batch_states)
            probs = normal(batch_actions, mu, sigma_sq)
            
            # ratio
            ratio = probs / (1e-15 + probs_old)
            
            # surrogate clip loss
            surr1 = ratio * torch.cat([batch_advantages]*num_outputs,1) # surrogate from conservative policy iteration
            surr2 = ratio.clamp(1-params.clip, 1+params.clip) * torch.cat([batch_advantages]*num_outputs,1)
            loss_clip = -torch.mean(torch.min(surr1, surr2))
            # should this be a mean along axis 0?
            
            # state-value function loss, do we even need this if they don't share params?
            vfloss1 = (v_pred - batch_returns)**2
            v_pred_clipped = v_pred_old + (v_pred - v_pred_old).clamp(-params.clip, params.clip)
            vfloss2 = (v_pred_clipped - batch_returns)**2
            loss_value = 0.5 * torch.mean(torch.max(vfloss1, vfloss2))
            # should this be a mean along axis 0?
            
            # loss on entropy bonus to ensure sufficient exploration
            loss_ent = -params.ent_coeff*torch.mean(probs*torch.log(probs+1e-5))
            
            # total
            total_loss = (loss_clip + loss_value + loss_ent)
#             total_loss = (loss_clip - loss_value + loss_ent)
            av_loss += loss_value.data[0] / float(params.num_epoch)
            
            # before step, update old_model:
            model_old.load_state_dict(model.state_dict())
            
            # step
            optimizer.zero_grad()
            total_loss.backward(retain_variables=True)
            optimizer.step()
        
        # t finish, print:
        df_infos = pd.DataFrame(infos)
        
        # show stats?
#         display(df_infos[["cash_bias","return","portfolio_value","market_value"]].describe().loc[["min","mean","max"]])
        
        report=OrderedDict(
            episode=episode,
#             reward=av_reward / float(cum_done),
            loss=av_loss,
            cash_bias=df_infos.cash_bias.mean(),
            market_value=df_infos.market_value.mean(),
            portfolio_value=df_infos.portfolio_value.mean(),
            reward=df_infos.reward.mean()
        )
        
        s = ', '.join(['{}={:2.4g}'.format(key,value) for key,value in report.items()])
        print(s)
        
        reports.append(report)
    
        memory.clear()
        torch.save(model_old, save_path)


episode=15, loss=9.719e-06, cash_bias=0.1576, market_value=1.023, portfolio_value=1.036, reward=-5.649e-06


/home/isisilon/.pyenv/versions/3.6.0/envs/jupyter3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type EIIE_CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


episode=31, loss=0.0001381, cash_bias=0.1966, market_value=1.042, portfolio_value=1.052, reward=7.133e-06
episode=47, loss=0.0002781, cash_bias=0.1989, market_value=1.042, portfolio_value=1.045, reward=-1.019e-06
episode=63, loss=0.0001417, cash_bias=0.2253, market_value=1.051, portfolio_value=1.049, reward=6.361e-06
episode=79, loss=5.829e-05, cash_bias=0.1883, market_value=1.029, portfolio_value=1.033, reward=2.362e-05
episode=95, loss=3.899e-05, cash_bias=0.1925, market_value=1.051, portfolio_value=1.069, reward=3.269e-06
episode=111, loss=4.012e-05, cash_bias=0.1828, market_value=1.059, portfolio_value=1.059, reward=2.533e-06
episode=127, loss=4.866e-05, cash_bias=0.3374, market_value=1.016, portfolio_value=1.034, reward=5.887e-06
episode=143, loss=5.706e-05, cash_bias=0.2415, market_value=1.044, portfolio_value=1.03, reward=1.553e-05
episode=159, loss=4.973e-05, cash_bias=0.1724, market_value=1.036, portfolio_value=1.042, reward=1.092e-05
episode=175, loss=4.329e-05, cash_bias=0.2

In [ ]:
# show progress
df=pd.DataFrame(reports)
g = sns.jointplot(x="episode", y="loss", data=df, kind="reg", size=10)
plt.show()

In [ ]:
# env_test = env

# Test
model.train(False)
state = env_test.reset()
for i in range(250):
    state = Variable(torch.Tensor(state).unsqueeze(0))
    mu, sigma_sq, v = model(state)
    eps = torch.randn(mu.size())
    action = (mu + sigma_sq.sqrt() * Variable(eps))
    env_action = action.data.squeeze().numpy()
    state, reward, done, info = env_test.step(env_action)
    if done:
        break
        
env.plot()